In [1]:
# Gerekli kütüphaneleri içe aktar
from io import open
import unicodedata
import string
import re
import random
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab, build_vocab_from_iterator
from collections import Counter 
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import pickle
import spacy

C:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
C:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [9]:
# gRPC üzerinden Zemberek dil işleme servislerini kullanmak için gerekli modülleri ve paketleri içe aktar
import sys
import grpc
import zemberek_grpc.language_id_pb2 as z_langid
import zemberek_grpc.language_id_pb2_grpc as z_langid_g
import zemberek_grpc.normalization_pb2 as z_normalization
import zemberek_grpc.normalization_pb2_grpc as z_normalization_g
import zemberek_grpc.preprocess_pb2 as z_preprocess
import zemberek_grpc.preprocess_pb2_grpc as z_preprocess_g
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g

# gRPC kanalını belirtilen adres ve port üzerinden oluştur
channel = grpc.insecure_channel('localhost:6789')

# Dil tespiti için servis istemcisini oluştur
langid_stub = z_langid_g.LanguageIdServiceStub(channel)

# Normalizasyon için servis istemcisini oluştur
normalization_stub = z_normalization_g.NormalizationServiceStub(channel)

# Metin ön işleme için servis istemcisini oluştur
preprocess_stub = z_preprocess_g.PreprocessingServiceStub(channel)

# Morfoloji analizi için servis istemcisini oluştur
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

# Dil tespiti fonksiyonu
def find_lang_id(i):
    response = langid_stub.Detect(z_langid.LanguageIdRequest(input=i))
    return response.langId

# Metni token'lara ayıran fonksiyon
def tokenize(i):
    response = preprocess_stub.Tokenize(z_preprocess.TokenizationRequest(input=i))
    return response.tokens

# Decode işlemini gerçekleştiren fonksiyon
def fix_decode(text):
    """Pass decode."""
    if sys.version_info < (3, 0):
        return text.decode('utf-8')
    else:
        return text
    
# Morfoloji analizi fonksiyonu
def analyze(i):
    response = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=i))
    return response;


In [88]:
# Metni normalize etmek için bir fonksiyon
def normalizeString(s):
    # Metni düşük harfe dönüştür, Türkçe karakterleri ve belirli noktalama işaretlerini koru
    s = s.lower().strip()
    # Sadece harfler, nokta (.), soru işareti (?), ünlem işareti (!) ve Türkçe karakterleri koru
    s = re.sub(r"[^a-zçğıöşü.!?,'']+", " ", s)
    return s

# Belirli kriterlere göre çiftleri filtrelemek için bir fonksiyon
def filterPair(p, max_length, prefixes):
    # Her iki cümle de belirtilen maksimum uzunluktan daha kısa mı kontrol et
    good_length = (len(p[0].split(' ')) < max_length) and (len(p[1].split(' ')) < max_length)
    # Eğer önekler belirtilmişse, cümlenin önek ile başlayıp başlamadığını kontrol et
    if len(prefixes) == 0:
        return good_length
    else:
        return good_length and p[0].startswith(prefixes)

# Belirli kriterlere göre çiftleri filtrelemek için bir fonksiyon
def filterPairs(pairs, max_length, prefixes=()):
    return [pair for pair in pairs if filterPair(pair, max_length, prefixes)]

# Veriyi hazırlamak için bir fonksiyon
def prepareData(lines, filter=False, reverse=False, max_length=10, prefixes=()):
    # Her bir satırı normalize et ve çiftlere ayır
    pairs = [(normalizeString(pair[0]), normalizeString(pair[1])) for pair in ceviriler]

    print(f"Given {len(pairs):,} sentence pairs.")

    # Eğer filtreleme etkinse, çiftleri belirtilen kriterlere göre filtrele
    if filter:
        pairs = filterPairs(pairs, max_length=max_length, prefixes=prefixes)
        print(f"After filtering, {len(pairs):,} remain.")

    return pairs


In [89]:
normalizeString("Mary'yi")

"mary'yi"

In [90]:
tokenize_tr(normalizeString("mary'yi"))

["mary'yi"]

['ingilizce']

In [91]:
with open("ceviriler/ceviriler.pkl", "rb") as f:
    ceviriler = pickle.load(f)

In [33]:
basic_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ",
    'are you', 'am i ', 
    'were you', 'was i ', 
    'where are', 'where is',
    'what is', 'what are'
)

In [51]:
ceviriler[5][1]

'Yasama organı, yasaları yapma sorumluluğuna sahiptir.'

In [92]:
pairss = [(normalizeString(pair[0]), normalizeString(pair[1])) for pair in ceviriler]

In [56]:
with open("ceviriler/pairss.pkl", "wb") as f2:
    pickle.dump(pairss, f2)

In [93]:
# İngilizce dil modelini yükleme
en_nlp = spacy.load("en_core_web_sm")

# İngilizce cümleleri token'lara ayıran fonksiyon
def tokenize_en(text):
    return [tok.text for tok in en_nlp.tokenizer(text)]

# Türkçe cümleleri token'lara ayıran fonksiyon
def tokenize_tr(sentence):
    liste = []
    # Zemberek dil işleme servisini kullanarak Türkçe cümleyi analiz etme
    analysis_result = analyze(sentence)
    for a in analysis_result.results:
        best = a.best
        lemmas = ""
        liste.append(a.token)
    
    return liste

In [97]:
# İngilizce ve Türkçe kelimeleri saymak için sayaçlar oluştur
en_counter = Counter()
tr_counter = Counter()

# Rastgele seçilmiş 5 çift üzerinde işlem yap
for eng, tur in random.choices(pairss, k=5):
    # İngilizce cümleyi ekrana yazdır
    print(f"English:  {eng}")
    # İngilizce cümleyi token'lara ayır ve ekrana yazdır
    print(tokenize_en(eng))
    # Türkçe cümleyi ekrana yazdır
    print(f"Turkish:  {tur}")
    # Türkçe cümleyi token'lara ayır ve ekrana yazdır
    aa = tokenize_tr(tur)
    print(aa)
    print()

    # İngilizce kelimeleri say
    en_counter.update(tokenize_en(eng))
    # Türkçe kelimeleri say
    tr_counter.update(aa)

English:  the camera detected motion and triggered the security system to alert the homeowners.
['the', 'camera', 'detected', 'motion', 'and', 'triggered', 'the', 'security', 'system', 'to', 'alert', 'the', 'homeowners', '.']
Turkish:  kamera, hareketi algıladı ve güvenlik sistemini ev sahiplerini uyaracak şekilde tetikledi.
['kamera', ',', 'hareketi', 'algıladı', 've', 'güvenlik', 'sistemini', 'ev', 'sahiplerini', 'uyaracak', 'şekilde', 'tetikledi', '.']

English:  the broadcaster delivered the news to a wide audience through the television network.
['the', 'broadcaster', 'delivered', 'the', 'news', 'to', 'a', 'wide', 'audience', 'through', 'the', 'television', 'network', '.']
Turkish:  yayıncı, televizyon ağı aracılığıyla geniş bir izleyici kitlesine haberleri iletti.
['yayıncı', ',', 'televizyon', 'ağı', 'aracılığıyla', 'geniş', 'bir', 'izleyici', 'kitlesine', 'haberleri', 'iletti', '.']

English:  i need to make a decision about which university to attend.
['i', 'need', 'to', 'make

In [98]:
# Özel token'ları tanımla
SPECIALS = ['<unk>', '<pad>', '<bos>', '<eos>']

# İngilizce ve Türkçe cümle listeleri
en_list = []
tr_list = []

# İngilizce ve Türkçe kelime sayaçları
en_counter = Counter()
tr_counter = Counter()

# İngilizce ve Türkçe cümle uzunlukları
en_lengths = []
tr_lengths = []

# Tokenleme işlemi
sayac = 0
for en, tr in pairss:
    # İngilizce ve Türkçe cümleleri token'lara ayır
    en_toks = tokenize_en(en)
    tr_toks = tokenize_tr(tr)
    
    # Token'ları ilgili listelere ekle
    en_list += [en_toks]
    tr_list += [tr_toks]
    
    # Kelime sayılarını güncelle
    en_counter.update(en_toks)
    tr_counter.update(tr_toks)
    
    # Cümle uzunluklarını kaydet
    en_lengths.append(len(en_toks))
    tr_lengths.append(len(tr_toks))
    
    sayac += 1
    
    # Her 1000 çift için ilerlemeyi ekrana yazdır
    if sayac % 1000 == 0:
        print(sayac)

# İngilizce ve Türkçe kelime dağarcıklarını oluştur
en_vocab = build_vocab_from_iterator(en_list, specials=SPECIALS)
tr_vocab = build_vocab_from_iterator(tr_list, specials=SPECIALS)


1000


In [99]:
datadir = "ceviriler"
with open(os.path.join(datadir,'en_lengths.pkl'), 'wb') as f:
    pickle.dump(en_lengths, f)
    
with open(os.path.join(datadir,'tr_lengths.pkl'), 'wb') as f:
    pickle.dump(tr_lengths, f)
    
with open(os.path.join(datadir,'en_counter.pkl'), 'wb') as f:
    pickle.dump(en_counter, f)
    
    
with open(os.path.join(datadir,'tr_counter.pkl'), 'wb') as f:
    pickle.dump(tr_counter, f)

In [100]:
# Veri setini bölme oranları
VALID_PCT = 0.1
TEST_PCT = 0.1

# Boş veri setleri oluştur
train_data = []
valid_data = []
test_data = []

# Rastgele tohum belirleme
random.seed(6547)

# Her bir çifti işleme al
sayac = 0
for (en, tr) in pairss:
    # İngilizce ve Türkçe cümleleri tensor'a çevir
    en_tensor_ = torch.tensor([en_vocab[token] for token in tokenize_en(en)])
    tr_tensor_ = torch.tensor([tr_vocab[token] for token in tokenize_tr(tr)])
    
    # Rastgele bir sayı çek ve bölme oranlarına göre veri setlerine ekle
    random_draw = random.random()
    if random_draw <= VALID_PCT:
        valid_data.append((en_tensor_, tr_tensor_))
    elif random_draw <= VALID_PCT + TEST_PCT:
        test_data.append((en_tensor_, tr_tensor_))
    else:
        train_data.append((en_tensor_, tr_tensor_))
    
    sayac += 1
    
    # Her 1000 çift için ilerlemeyi ekrana yazdır
    if sayac % 1000 == 0:
        print(sayac)

# Bölünmüş veri seti boyutlarını ekrana yazdır
print(f"""
      Training pairs: {len(train_data):,}
      Validation pairs: {len(valid_data):,}
      Test pairs: {len(test_data):,}""")


1000

      Training pairs: 1,568
      Validation pairs: 155
      Test pairs: 207


In [101]:
# Özel token indekslerini belirle
PAD_IDX = en_vocab['<pad>']
BOS_IDX = en_vocab['<bos>']
EOS_IDX = en_vocab['<eos>']

# İki dilin özel tokenlerinin indekslerini karşılaştır ve eşit olup olmadığını kontrol et
for en_id, tr_id in zip(en_vocab.lookup_indices(SPECIALS), tr_vocab.lookup_indices(SPECIALS)):
    assert en_id == tr_id

In [102]:
def generate_batch(data_batch):
    '''
    Veri yığınlarını modelleme için hazırlar. Her bir örneğe BOS/EOS belirteçlerini ekler, tensörleri birleştirir
    ve daha kısa cümlelerin sonundaki boşlukları <pad> belirteci ile doldurur. 
    English-to-Turkish DataLoader'ında collate_fn olarak kullanılması amaçlanmıştır.

    Input:
    - data_batch, yukarıda oluşturulan veri setlerinden alınan (İngilizce, Türkçe) tuple'larını içeren bir iterasyon

    Output:
    - en_batch: İngilizce token ID'leri içeren (maksimum uzunluk X yığın boyutu) bir tensör
    - tr_batch: Türkçe token ID'leri içeren (maksimum uzunluk X yığın boyutu) bir tensör 
    '''
    
    en_batch, tr_batch = [], []
    
    for (en_item, tr_item) in data_batch:
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
        tr_batch.append(torch.cat([torch.tensor([BOS_IDX]), tr_item, torch.tensor([EOS_IDX])], dim=0))

    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX, batch_first=False)
    tr_batch = pad_sequence(tr_batch, padding_value=PAD_IDX, batch_first=False)

    return en_batch, tr_batch


In [103]:
# Mini-batch boyutunu belirle
BATCH_SIZE = 16

# DataLoader ile eğitim, doğrulama ve test veri iteratörlerini oluştur
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)


In [104]:
datadir = "ceviriler"

# İngilizce kelime dağarcığını pickle formatında kaydetme
with open(os.path.join(datadir,'en_vocab.pkl'), 'wb') as f:
    pickle.dump(en_vocab, f)

# Türkçe kelime dağarcığını pickle formatında kaydetme
with open(os.path.join(datadir,'tr_vocab.pkl'), 'wb') as f:
    pickle.dump(tr_vocab, f)

# Eğitim veri setini pickle formatında kaydetme
with open(os.path.join(datadir,'train_data.pkl'), 'wb') as f:
    pickle.dump(train_data, f)
    
# Doğrulama veri setini pickle formatında kaydetme
with open(os.path.join(datadir,'valid_data.pkl'), 'wb') as f:
    pickle.dump(valid_data, f)

# Test veri setini pickle formatında kaydetme
with open(os.path.join(datadir,'test_data.pkl'), 'wb') as f:
    pickle.dump(test_data, f)

# Eğitim veri seti yükleyicisini pickle formatında kaydetme
with open(os.path.join(datadir,'train_iter.pkl'), 'wb') as f:
    pickle.dump(train_iter, f)

# Test veri seti yükleyicisini pickle formatında kaydetme
with open(os.path.join(datadir, 'test_iter.pkl'), 'wb') as f:
    pickle.dump(test_iter, f)

# Doğrulama veri seti yükleyicisini pickle formatında kaydetme
with open(os.path.join(datadir, 'valid_iter.pkl'), 'wb') as f:
    pickle.dump(valid_iter, f)


In [105]:
# Eğitim veri iteratöründen örnekler al ve ekrana yazdır
for i, (en_id, tr_id) in enumerate(train_iter):
    print('English:', ' '.join([en_vocab.lookup_token(idx) for idx in en_id[:, 0]]))
    print('Turkish:', ' '.join([tr_vocab.lookup_token(idx) for idx in tr_id[:, 0]]))
    
    # İlk 5 mini-batch'i ekrana yazdıktan sonra döngüyü sonlandır
    if i == 4: 
        break
    else:
        print()


English: <bos> please provide your input on the proposed changes to the project . <eos> <pad> <pad> <pad> <pad> <pad> <pad>
Turkish: <bos> lütfen projedeki önerilen değişikliklerle ilgili görüşlerinizi belirtin . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

English: <bos>   the cat is sleeping peacefully on the sofa . <eos> <pad> <pad> <pad> <pad>
Turkish: <bos> kedi , koltukta huzurlu bir şekilde uyuyor . <eos> <pad> <pad> <pad> <pad> <pad> <pad>

English: <bos> the national flag is symbolic of the country 's identity and values . <eos> <pad> <pad>
Turkish: <bos> ulusal bayrak , ülkenin kimliği ve değerleri için semboliktir . <eos> <pad> <pad> <pad>

English: <bos> the councilor played a key role in making decisions for the local community . <eos> <pad> <pad> <pad> <pad>
Turkish: <bos> meclis üyesi , yerel toplum için kararlar almakta önemli bir rol oynadı . <eos> <pad> <pad>

English: <bos> the researchers were able to derive meaningful conclusions from the data . <eos> <pa

In [106]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout_p=0.1, max_len=100):
        super().__init__()
        
        self.dropout = nn.Dropout(dropout_p)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, num_attention_heads, 
                 num_encoder_layers, num_decoder_layers, dim_feedforward, 
                 max_seq_length, pos_dropout, transformer_dropout):
        super().__init__()
        self.d_model = d_model
        self.embed_src = nn.Embedding(input_dim, d_model)
        self.embed_tgt = nn.Embedding(output_dim, d_model)
        self.pos_enc = PositionalEncoding(d_model, pos_dropout, max_seq_length)
        
        self.transformer = nn.Transformer(d_model, num_attention_heads, num_encoder_layers, 
                                          num_decoder_layers, dim_feedforward, transformer_dropout)
        self.output = nn.Linear(d_model, output_dim)
        
    def forward(self,
                src=None, 
                tgt=None,
                src_mask=None,
                tgt_mask=None, 
                src_key_padding_mask=None, 
                tgt_key_padding_mask=None,
                memory_key_padding_mask=None,
                src_embeds=None, 
                tgt_embeds=None):
        
        if (src_embeds is None) and (src is not None):
            if (tgt_embeds is None) and (tgt is not None):
                src_embeds, tgt_embeds = self._embed_tokens(src, tgt)
        elif (src_embeds is not None) and (src is not None):
            raise ValueError("Must specify exactly one of src and src_embeds")
        elif (src_embeds is None) and (src is None):
            raise ValueError("Must specify exactly one of src and src_embeds")
        elif (tgt_embeds is not None) and (tgt is not None):
            raise ValueError("Must specify exactly one of tgt and tgt_embeds")
        elif (tgt_embeds is None) and (tgt is None):
            raise ValueError("Must specify exactly one of tgt and tgt_embeds")
        
        output = self.transformer(src_embeds, 
                                  tgt_embeds, 
                                  tgt_mask=tgt_mask, 
                                  src_key_padding_mask=src_key_padding_mask,
                                  tgt_key_padding_mask=tgt_key_padding_mask,
                                  memory_key_padding_mask=memory_key_padding_mask)
        
        return self.output(output)
    
    def _embed_tokens(self, src, tgt):
        src_embeds = self.embed_src(src) * np.sqrt(self.d_model)
        tgt_embeds = self.embed_tgt(tgt) * np.sqrt(self.d_model)
        
        src_embeds = self.pos_enc(src_embeds)
        tgt_embeds = self.pos_enc(tgt_embeds)
        return src_embeds, tgt_embeds

In [107]:
transformer = TransformerModel(input_dim=len(en_vocab), 
                             output_dim=len(tr_vocab), 
                             d_model=256, 
                             num_attention_heads=8,
                             num_encoder_layers=6, 
                             num_decoder_layers=6, 
                             dim_feedforward=2048,
                             max_seq_length=32,
                             pos_dropout=0.15,
                             transformer_dropout=0.3)

transformer = transformer.to("cpu")

C:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [108]:
def predict_transformer(text, model, 
                        src_vocab=en_vocab, 
                        src_tokenizer=tokenize_en, 
                        tgt_vocab=tr_vocab, 
                        device="cpu"):
    
    input_ids = [src_vocab[token.lower()] for token in src_tokenizer(text)]
    input_ids = [BOS_IDX] + input_ids + [EOS_IDX]
    
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_ids).to(device).unsqueeze(1) 
        
        causal_out = torch.ones(MAX_SENTENCE_LENGTH, 1).long().to(device) * BOS_IDX
        for t in range(1, MAX_SENTENCE_LENGTH):
            decoder_output = transformer(input_tensor, causal_out[:t, :])[-1, :, :]
            next_token = decoder_output.data.topk(1)[1].squeeze()
            causal_out[t, :] = next_token
            if next_token.item() == EOS_IDX:
                break
                
        pred_words = [tgt_vocab.lookup_token(tok.item()) for tok in causal_out.squeeze(1)[1:(t)]]
        return " ".join(pred_words)

In [109]:
def train_transformer(model, iterator, optimizer, loss_fn, device, clip=None):
    model.train()
        
    epoch_loss = 0
    with tqdm(total=len(iterator), leave=False) as t:
        for i, (src, tgt) in enumerate(iterator):
            src = src.to(device)
            tgt = tgt.to(device)
            
            # Create tgt_inp and tgt_out (which is tgt_inp but shifted by 1)
            tgt_inp, tgt_out = tgt[:-1, :], tgt[1:, :]

            tgt_mask = model.transformer.generate_square_subsequent_mask(tgt_inp.size(0)).to(device)
            src_key_padding_mask = (src == PAD_IDX).transpose(0, 1)
            tgt_key_padding_mask = (tgt_inp == PAD_IDX).transpose(0, 1)
            memory_key_padding_mask = src_key_padding_mask.clone()
            
            optimizer.zero_grad()
            
            output = model(src=src, tgt=tgt_inp, 
                           tgt_mask=tgt_mask,
                           src_key_padding_mask = src_key_padding_mask,
                           tgt_key_padding_mask = tgt_key_padding_mask,
                           memory_key_padding_mask = memory_key_padding_mask)
            
            loss = loss_fn(output.view(-1, output.shape[2]),
                           tgt_out.view(-1))
            
            loss.backward()
            
            if clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step()
            epoch_loss += loss.item()
            
            avg_loss = epoch_loss / (i+1)
            t.set_postfix(loss='{:05.3f}'.format(avg_loss),
                          ppl='{:05.3f}'.format(np.exp(avg_loss)))
            t.update()
            
    return epoch_loss / len(iterator)
    
def evaluate_transformer(model, iterator, loss_fn, device):
    model.eval()
        
    epoch_loss = 0
    with torch.no_grad():
        with tqdm(total=len(iterator), leave=False) as t:
            for i, (src, tgt) in enumerate(iterator):
                src = src.to(device)
                tgt = tgt.to(device)
                
                # Create tgt_inp and tgt_out (which is tgt_inp but shifted by 1)
                tgt_inp, tgt_out = tgt[:-1, :], tgt[1:, :]
                
                tgt_mask = model.transformer.generate_square_subsequent_mask(tgt_inp.size(0)).to(device)
                src_key_padding_mask = (src == PAD_IDX).transpose(0, 1)
                tgt_key_padding_mask = (tgt_inp == PAD_IDX).transpose(0, 1)
                memory_key_padding_mask = src_key_padding_mask.clone()

                output = model(src=src, tgt=tgt_inp, 
                               tgt_mask=tgt_mask,
                               src_key_padding_mask = src_key_padding_mask,
                               tgt_key_padding_mask = tgt_key_padding_mask,
                               memory_key_padding_mask = memory_key_padding_mask)
                
                loss = loss_fn(output.view(-1, output.shape[2]),
                               tgt_out.view(-1))
                
                epoch_loss += loss.item()
                
                avg_loss = epoch_loss / (i+1)
                t.set_postfix(loss='{:05.3f}'.format(avg_loss),
                              ppl='{:05.3f}'.format(np.exp(avg_loss)))
                t.update()
    
    return epoch_loss / len(iterator)



In [110]:
xf_optim = torch.optim.AdamW(transformer.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

N_EPOCHS = 50
CLIP = 15 # clipping value, or None to prevent gradient clipping
EARLY_STOPPING_EPOCHS = 5
SAVE_DIR = os.getcwd() 
model_path = os.path.join(SAVE_DIR, 'transformer_en_tr.pt')
transformer_metrics = {}
best_valid_loss = float("inf")
early_stopping_count = 0
for epoch in tqdm(range(N_EPOCHS), desc="Epoch"):
    train_loss = train_transformer(transformer, train_iter, xf_optim, loss_fn, device, clip=CLIP)
    valid_loss = evaluate_transformer(transformer, valid_iter, loss_fn, device)
    
    if valid_loss < best_valid_loss:
        tqdm.write(f"Checkpointing at epoch {epoch + 1}")
        best_valid_loss = valid_loss
        torch.save(transformer.state_dict(), model_path)
        early_stopping_count = 0
    elif epoch > EARLY_STOPPING_EPOCHS:
        early_stopping_count += 1
    
    transformer_metrics[epoch+1] = dict(
        train_loss = train_loss,
        train_ppl = np.exp(train_loss),
        valid_loss = valid_loss,
        valid_ppl = np.exp(valid_loss)
    )
    
    if early_stopping_count == EARLY_STOPPING_EPOCHS:
        tqdm.write(f"Early stopping triggered in epoch {epoch + 1}")
        break

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

C:\Users\VuralBayrakli\anaconda3\envs\env\lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
